In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [4]:
df = df.sort_values("date")

In [6]:
df = df.reset_index(drop=True)

In [8]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [10]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)


In [11]:
df[df["team"] == "WAS"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,seasons,date,won,target
24,240.0,42.0,97.0,0.433,6.0,22.0,0.273,30.0,38.0,0.789,...,31.2,148.0,125.0,PHI,115,0,2018,2017-10-18,True,True
51,240.0,40.0,75.0,0.533,6.0,17.0,0.353,29.0,35.0,0.829,...,43.6,215.0,122.0,DET,111,0,2018,2017-10-20,True,True
88,240.0,37.0,84.0,0.440,9.0,30.0,0.300,26.0,28.0,0.929,...,35.1,272.0,115.0,DEN,104,1,2018,2017-10-23,True,False
119,265.0,39.0,95.0,0.411,6.0,26.0,0.231,15.0,23.0,0.652,...,32.9,129.0,98.0,LAL,102,1,2018,2017-10-25,False,False
150,240.0,43.0,97.0,0.443,15.0,33.0,0.455,16.0,22.0,0.727,...,30.9,210.0,122.0,GSW,120,1,2018,2017-10-27,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17465,240.0,38.0,88.0,0.432,5.0,33.0,0.152,21.0,30.0,0.700,...,31.5,144.0,114.0,POR,108,0,2024,2024-04-05,False,False
17507,240.0,42.0,89.0,0.472,16.0,40.0,0.400,22.0,30.0,0.733,...,30.4,145.0,120.0,TOR,130,1,2024,2024-04-07,False,False
17528,240.0,44.0,89.0,0.494,20.0,43.0,0.465,13.0,17.0,0.765,...,72.4,199.0,131.0,MIN,130,1,2024,2024-04-09,False,False
17568,240.0,45.0,86.0,0.523,18.0,43.0,0.419,19.0,29.0,0.655,...,29.3,159.0,127.0,CHI,129,0,2024,2024-04-12,False,False


In [13]:
df["target"][pd.isnull(df["target"])] = 2

/var/folders/bq/6877_wbj2zgchbp1mq0cb1zw0000gn/T/ipykernel_67733/1618397414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [14]:
df["target"] = df["target"].astype(int, errors="ignore")

In [19]:
df["target"].value_counts()

1    8885
0    8885
2      30
Name: target, dtype: int64

In [24]:
nulls = pd.isnull(df)
nulls = nulls.sum()

In [25]:
nulls = nulls[nulls > 0]

In [26]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [27]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'orb',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'seasons', 'date', 'won', 'target'],
      dtype='object', length=138)

In [28]:
 df = df[valid_columns].copy()

In [34]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction='forward', cv=split)

In [35]:
removed_columns = ["season", "date", 'won', 'target', 'team', 'team_opp']

In [36]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [37]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [ ]:
sfs.fit(df[selected_columns], df["target"])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])
predictors